<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2024/blob/main/LLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from scipy.optimize import minimize

In [ ]:
p = Props(['Ethanol', 'Water', 'Toluene'])

In [ ]:
#dSsys + dSsurr >= 0
#dS = dqrev/T

#dU = dq + dW
#dU = dq - P dV

# dW = F dx = - Pext dV
# dW = -Pext dV

#dU = dqrev - P dV
#dSsys + dSsurr >= 0
#dS - dqrev/T >= 0

#dS - (dU + P dV)/T >= 0
#Gibbs energy const T, P
# T dS - (dU + P dV) >= 0
# d(TS) - dU + d(PV) >= 0
# d(TS) - dH >=0
#d(H - TS) <=0
# dG <= 0


#G = nL*GL + nV*Gv
#dG = nL*dGL + GL*dnL + nV*dGv + dnV*GV
# dnL = - dnV
#dG = nL*dGL + nV*dGv + dnL*(GL - GV)
#dGL

#dU = T dS - P dV
#dH = d(U + PV) = T dS - P dV + P dV + V dP = T dS + V dP
#dG = d(H-TS) = dH - TdS - S dT = dH - TdS - SdT = T dS + V dP - TdS - SdT =   -S dT + V dP
#dG = -S dT + V dP

#const T, P
#dG = nL*dGL + nV*dGv + dnL*(GL - GV) = dnL*(GL - GV) = 0 ====> GL = GV

#chemical potential of each component is equal in the phases in equilibrium
#ui = d(nG)/dni

In [ ]:
T = 3
F = 1. * np.array([0.1, 0.45, 0.45])
L1guess = np.array([  0.05 , 0.01, 0.44])
L2guess = F - L1guess
vec_guess = np.r_[L1guess, L2guess]

In [ ]:
def G(vec):
    L1, L2 = np.split(vec, 2)
    x1, x2 = L1/np.sum(L1), L2/np.sum(L2)
    gamma1 = p.NRTL_gamma(x1, T)
    gamma2 = p.NRTL_gamma(x2, T)
    return np.sum(L1 * np.log(x1*gamma1)) + np.sum(L2*np.log(x2*gamma2))

In [ ]:
def MB(vec):
    L1, L2 = np.split(vec, 2)
    return L1+L2 - F

In [ ]:
sol = minimize(G, vec_guess, bounds=[(1e-12, F[0]), (1e-12, F[1]), (1e-12, F[2])]*2, constraints = dict(type='eq',fun=MB)).x

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_slsqp_py.py:495: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  a_eq = vstack([con['jac'](x, *con['args'])


In [ ]:
L1_sol, L2_sol = np.split(sol,2)
L1_tot, L2_tot = np.sum(L1_sol), np.sum(L2_sol)

In [ ]:
L1_tot

0.47399478086142516

In [ ]:
L2_tot

0.5260052191385748

In [ ]:
L1_sol/L1_tot

array([5.17546480e-02, 1.93995521e-04, 9.48051356e-01])

In [ ]:
L2_sol/L2_tot

array([0.14347494, 0.85533001, 0.00119505])